In [108]:
import pandas as pd

test_set = pd.read_csv("testset.csv")
test_set.rename(columns={'Unnamed: 0': 'packet #'}, inplace=True)
test_set.rename(columns={'source_text': 'Query'}, inplace=True)
test_set.rename(columns={'target_text': 'Response'}, inplace=True)

save_client_data = []
counters = []
counter_invalid = 0
counter_valid = 0
def packet_chunks(query, dataFromServer):
    hex_chunks_query = [query[i:i + 2] for i in range(0, len(query), 2)] #print(f"hex_chunks_query is {hex_chunks_query}, len is {len(hex_chunks_query)}")
    hex_chunks_response = [dataFromServer[i:i + 2] for i in range(0, len(dataFromServer), 2)] #print(f"hex_chunks_response is {hex_chunks_response}, len is {len(hex_chunks_response)}")
    # DIVIDE Header and payload
    query_header = hex_chunks_query[:7] #print(f"query_header is {query_header}, len is {len(query_header)}")
    query_payload = hex_chunks_query[7:] #print(f"query_payload is {query_payload}, len is {len(query_payload)}")
    response_header = hex_chunks_response[:7] #print(f"response_header is {response_header}, len is {len(response_header)}")
    response_payload = hex_chunks_response[7:] #print(f"response_payload is {response_payload}, len is {len(response_payload)}")
    return query_header, query_payload, response_header, response_payload 


def check_header_IDs(query_header, response_header, query_payload):
    print("1. Checking the header of the packet")
    #1 Check the TID (1st 2bytes) and PID (2nd 2bytes) and UID (last 1byte)
    if query_header[:4] == response_header[:4] and query_header[-1] == response_header[-1]:
        print("The TID, PID, UID are correct")
    else:
        print("The TID and/or PID and/or UID are not correct")
    #2 Check the length of the header 
    if int(query_header[5], base=16) == (len(query_payload)+1):
        print("The length of the query is = to the payload + the UID")
        print("The header of the packet is valid up to the FC code")

def check_payload(query_payload, response_payload, fc, query_header=1): #query_header only in FC15
    print("Checking the FC code of the packet")
    if query_payload[0] == response_payload[0]:
        print("The FC code is correct")
        
    #check the payload for each case
    if fc == "01":
        bit_count = int(query_payload[-1], base=16)
        byte_count = int(response_payload[1], base=16)
        if byte_count == (int(bit_count/8)+1):
            print("The bit count in the query is equal to the byte count in the response")
        
    if fc == "03": #NOTE: the reference (registers address) cant be verified - depicted on wireshark only
        #2 Check the Word Count in the query and the number of Registers in response (2bytes)
        word_count = int(query_payload[-1], base=16)
        num_registers = int(len(response_payload[2:])/2)
        if num_registers == word_count:
            print(f"The Word Count {word_count} in the query is correct wrt the number of registers in the response {num_registers}.")
        #3 Check the Byte Count in the response and the length/number of Registers in response
        byte_count = int(response_payload[1], base=16)
        length_regiters = len(response_payload[2:])
        if byte_count == length_regiters:
            print(f"The Byte Count {byte_count} in the response is correct wrt length of registers in the response {num_registers}.")
        print("The payload of the packet is valid wrt the query")
        print("Its a valid ModbusTCP traffic")

    if fc == "15": # Note the response payload = the query payload up to the Bit Count/ query has additional attributes Byte count and Data (1byte each) - 5chunks
        #2 Check the Reference Number
        if query_payload[1:3] == response_payload[1:3]:
            print("The reference number is correct")
        #3 Check the Bit Count
        if query_payload[3:5] == response_payload[3:5]:
            print("The bit count is correct")
        print("The response is valid")
        #4 Check the query byte count and data length
        if (len(query_payload) - len(query_payload[:6])) == int(query_header[5], base=16):
            print("The data length is as its mentioned in the byte count")
        print("The payload of the packet is valid wrt the query")
        print("Its a valid ModbusTCP traffic")
    
    if fc == "16":
        #2 Check the Reference Number
        if query_payload[1:3] == response_payload[1:3]:
            print("The reference number is correct")
        #3 Check the Word Count
        if query_payload[3:5] == response_payload[3:5]:
            print("The word count is correct")
        print("The response is valid")
        word_count = int(query_payload[4], base=16) #word count = number of registers
        num_registers = int(len(query_payload[6:])/2)
        if num_registers == word_count:
            print(f"The Word Count {word_count} in the query is correct wrt the number of registers in the query {num_registers}.")
        byte_count = int(query_payload[5], base=16) #byte count = number of bytes for all registers 
        bytes_registers = len(query_payload[6:])
        if bytes_registers == byte_count:
            print(f"The Byte Count {byte_count} in the query is correct wrt the bytes of registers in the query {bytes_registers}.")
        print("The payload of the packet is valid wrt the query")
        print("Its a valid ModbusTCP traffic")





for i in range(len(test_set)):
    query = test_set['Query'][i]
    #query = "edbd00000006000100180003"
    #query = "ede300000008000f005b000801ff"
    #query = "ede70000001700100036000810f816977f533106d35f2471bca50778b9"
    print('query is:', query)
    
    #dataFromServer = "e4970000000b000308fc1265ad0ccccccc" 
    #dataFromServer = "edbd0000000400010107"
    #dataFromServer = "ede300000006000f005b0008"
    #dataFromServer = "ede700000006001000360008"
    print("dataFromServer is:", dataFromServer)

    # Convert the hexadecimal string to a list of two-character chunks (Note: 2 bytes = 2 chunks)
    query_header, query_payload, response_header, response_payload = packet_chunks(query, dataFromServer)

    # validate the prediction response with the target response
    #if dataFromServer == test_set['Response'][i]:
        #print("The predicted response is EXACTLY the same as the target response!")
        
    #elif dataFromServer[0:15:1] == test_set['Response'][i][0:15:1]:  # upto 8bytes (8 chunks, 16 characters)
        #print("The predicted response is valid wrt the target response up to the FC code")        
                
    # FC01
    if response_payload[0] == "01":
        print("Its FC01: Reading Coils")
                                                            #Note: reference number cant be verified - depicted on wireshark only/ the Bit Count 
        check_header_IDs(query_header, response_header, query_payload)
        check_payload(query_payload, response_payload, response_payload[0])

    # FC03
    if response_payload[0] == "03":
        print("Its FC03: Reading Holding Registers")
                                                            
        check_header_IDs(query_header, response_header, query_payload)
        check_payload(query_payload, response_payload, response_payload[0])
            
    # FC05
    if response_payload[0] == "05":
        print("Its FC05: Write Single Coil")
        # check the response wrt the query
        if len(hex_chunks_response) == len(hex_chunks_query) and dataFromServer == test_set['Response'][i]:
            print("Its a valid ModbusTCP traffic")
        else:
            print("something is wrong")

    # FC15
    if response_payload[0] == "0f":
        print("Its FC15: Write Multiple Coils")

        check_header_IDs(query_header, response_header, query_payload)
        check_payload(query_payload, response_payload, response_payload[0], query_header)
        
    # FC16
    if response_payload[0] == "10":
        print("Its FC16: Write Multiple Registers")
        
        check_header_IDs(query_header, response_header, query_payload)               
        check_payload(query_payload, response_payload, response_payload[0])


query is: edbd00000006000100180003
dataFromServer is: edbd0000000400010107
hex_chunks_query is ['ed', 'bd', '00', '00', '00', '06', '00', '01', '00', '18', '00', '03'], len is 12
hex_chunks_response is ['ed', 'bd', '00', '00', '00', '04', '00', '01', '01', '07'], len is 10
query_header is ['ed', 'bd', '00', '00', '00', '06', '00'], len is 7
query_payload is ['01', '00', '18', '00', '03'], len is 5
response_header is ['ed', 'bd', '00', '00', '00', '04', '00'], len is 7
response_payload is ['01', '01', '07'], len is 3
Its FC01: Reading Coils
1. Checking the header of the packet
The TID, PID, UID are correct
The length of the query is = to the payload + the UID
The header of the packet is valid up to the FC code
2. Checking the payload of the packet
The FC code is correct


In [119]:
response_payload

['01', '01', '07']

In [ ]:
context = """Query: 0001000000040a010100  Response:0000000000060a08000a0000 """
question = "0001000000040a810500"

inputs = model.tokenizer([context],[question], return_tensors="pt")
output = tokenizer.decode(inputs['input_ids'][0])
answer = model.predict(output)

print("Predicted packet response is:", answer)